# retrieve the closest genes of the GOI 

first we need some post processing of the output of the WGCNA script

In [1]:
import pandas as pd

import os

# set the path
data_dir = os.path.join('data')
output_dir = os.path.join('output_TPM5_CV50_stem_power6') # renaming the output based on the parameters used

annot = pd.read_csv(os.path.join(data_dir, "lab_annotation_tremula.csv"), index_col="geneID")
connectivity_df = pd.read_csv(os.path.join(output_dir, "connectivity_table.csv"), index_col=0)
connectivity_df.join(annot).to_csv(os.path.join(output_dir, "connectivity_table_annot.csv"))

retrieve the goi related info

In [2]:
import pandas as pd

adj_df = pd.read_csv(os.path.join(output_dir, "adj_matrix.csv"), index_col=0)
k_df = pd.read_csv(os.path.join(output_dir, "connectivity_table_annot.csv"), index_col=0)

goi_list = [
    "PtXaTreH.14G131700",
    "PtXaTreH.10G125100",
    "PtXaTreH.06G010600",
    "PtXaTreH.05G120200",
    "PtXaTreH.03G064200",
    "PtXaTreH.02G086600",
]

In [3]:
adj_df = adj_df.reset_index()
adj_df.index = adj_df.columns

## basic stats info 

In [4]:
# Describe the selected columns with additional percentiles
percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

descriptive_stats = adj_df.describe(percentiles=percentiles)

# Display the descriptive statistics
descriptive_stats = descriptive_stats.T

descriptive_stats.loc[:,["mean","std","min","50%","60%","70%","80%","90%"]].join(k_df).to_csv("adj_stat_annot.csv")
descriptive_stats = descriptive_stats.loc[:,["mean","std","min","50%","60%","70%","80%","90%"]]
descriptive_stats = descriptive_stats.add_suffix("_PCC")

for thresdhold in percentiles:
    descriptive_stats[f"gene number PCC > {thresdhold}"] = (adj_df>thresdhold).sum()

descriptive_stats.join(k_df).to_csv(os.path.join(output_dir,"adj_stat_annot.csv"))

## export the most closely related genes based on a cutoff

In [5]:
threshold = 0.7

output_genelist_dir = os.path.join(output_dir, "adjacent_gene_list")
# create dir if not exist
if not os.path.exists(output_genelist_dir):
    os.makedirs(output_genelist_dir)

for goi in goi_list:
    # Extract correlated genes
    try:
        correlated_genes = adj_df.loc[goi][adj_df.loc[goi] > threshold].dropna()
        goi_module = k_df.loc[goi].Module
        if not correlated_genes.empty:
            # Create a DataFrame
            correlated_genes_df = pd.DataFrame(correlated_genes).reset_index()
            correlated_genes_df.columns = ['gene', 'adj_value']
            correlated_genes_df = correlated_genes_df.set_index('gene')
            # Join with annotation DataFrame
            result_df = correlated_genes_df.join(k_df)

            # Save to CSV
            result_df.to_csv(os.path.join(output_genelist_dir, f'{goi}_{goi_module}_correlated_genes.csv'))
    except:
        print(f"Error processing {goi}")
        continue

Error processing PtXaTreH.02G086600
